In [3]:
pip install requests
pip install beautifulSoup4
pip install selenium
pip install webdriver-manager
pip install wordcloud
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import time
import re
import sys
from selenium import webdriver
from bs4 import BeautifulSoup
import csv
from urllib.request import urlopen
from wordcloud import WordCloud
import matplotlib.pyplot as plt

driver = webdriver.Chrome('chromedriver_win32/chromedriver.exe')

with open ("Stabucks_drink.csv", 'w', encoding='utf-8',newline="") as fileWrite:
    myWriter = csv.writer(fileWrite)
    driver.get("https://www.starbucks.co.kr/menu/drink_list.do")
    time.sleep(7)

    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    
    drinks = soup.findAll('li',{'class':'menuDataSet'})
    
    #Stabucks_menu.csv파일에 스타벅스 음료 리스트 저장 
    for row in drinks:
        img_tag = row.find("img")
        name = img_tag['alt']
        myWriter.writerow(name.replace(" ","").strip('\n').split(','))

with open ("Stabucks_cake.csv", 'w', encoding='utf-8',newline="") as fileWrite:
    myWriter = csv.writer(fileWrite)
    driver.get("https://www.starbucks.co.kr/menu/food_list.do?CATE_CD=product_cake")
    time.sleep(7)

    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    
    drinks = soup.findAll('li',{'class':'menuDataSet'})
    driver.quit()
    
    #Stabucks_menu.csv파일에 스타벅스 케이크 리스트 저장 
    for row in drinks:
        img_tag = row.find("img")
        name = img_tag['alt']
        myWriter.writerow(name.replace(" ","").strip('\n').split(','))

while True:   
    print("1. 스타벅스 음료 \n2. 스타벅스 케이크")
    n = int(input("찾고싶은 정보 번호 입력 >"))
    if n==1:
        with open('Stabucks_drink.csv', 'r',encoding="utf-8") as fileRead:
            fileMatrix=[]
            for line in fileRead:
                fileMatrix.append(line.strip("\n").split(","))
        break
    elif n==2:
        with open('Stabucks_cake.csv', 'r',encoding="utf-8") as fileRead:
            fileMatrix=[]
            n=0
            for line in fileRead:
                fileMatrix.append(line.strip("\n").split(","))
        break
    else:
        print("잘못 입력하셨습니다.")
        print()

print()           
print("*** 스타벅스 메뉴목록 ***")
print()
for i in range(len(fileMatrix)):
    print(str(i+1) + "." + str(fileMatrix[i]))

print()
choice_num = int(input("원하는 메뉴 번호 입력 >"))
choice_menu = str(fileMatrix[choice_num-1]).strip("[]").replace(" ","").strip("''")

#메뉴 이름안에 "스타벅스" 들어가지 않으면 해시태그 검색할때 앞에 스타벅스 붙여주기 
if choice_menu.find("스타벅스")== -1:
    choice_menu="스타벅스"+choice_menu
#메뉴 이름에 190ML들어가면, 제거
if choice_menu.find("190ML")!=-1:
    choice_menu=choice_menu.replace("190ML","")
#메뉴 이름에 &들어가면 제거
if choice_menu.find("&")!=-1:
    choice_menu=choice_menu.replace("&","")
#메뉴 이름에 米들어가면 제거    
if choice_menu.find("米")!=-1:
    choice_menu=choice_menu.replace("米","")
#메뉴 이름에 New들어가면 제거   
if choice_menu.find("New")!=-1:
    choice_menu=choice_menu.replace("New","")
#메뉴 이름에 괄호 들어가면 제거
if choice_menu.find("(")!=-1:
    choice_menu=choice_menu.replace("(","")
if choice_menu.find(")")!=-1:
    choice_menu=choice_menu.replace(")","")

driver = webdriver.Chrome('chromedriver_win32/chromedriver.exe')
driver.get('http://www.instargram.com')
time.sleep(3)

#인스타그램 로그인
email = 'ksy991018@naver.com'  
input_id = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[0]
input_id.clear()
input_id.send_keys(email)

password = 'sylucy12' 
input_pw = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[1]
input_pw.clear()
input_pw.send_keys(password)
input_pw.submit()
time.sleep(3)

#해시태그 검색 
url = "https://www.instagram.com/explore/tags/{}/".format(choice_menu)

driver.get(url)
time.sleep(2)

try: 
    error=0
    data = driver.find_element_by_css_selector('.g47SY').text
    print("검색 량 : " + data)
    
    #ex)1,494로 된거 쉼표제거한 후 int형태인 1494로 바꿔주기
    data_num=int(data.replace(",",""))

    #현재 directory내 insta_img 폴더 안에 검색한 해시태그 관련 사진들 저장
    #단, 검색했을때 뜨는 사진만 저장
    
    def image():
        html=driver.page_source
        soup = BeautifulSoup(html)
        insta = soup.select('.v1Nh3.kIKUG._bz0w')
        count = 0
        for i in insta:
            imgURL = i.select_one('.KL4Bh').img['src']
            with urlopen(imgURL) as f:
                count +=1
                with open('./insta_img/'+ choice_menu + str(count)+ '.jpg',"wb") as h:
                    img=f.read()
                    h.write(img)
        if count>0:
            return 1
        else:
            return 0

    if image():
        print("\n#{0} 인스타그램 사진 저장 완료!".format(choice_menu))
    else:
        print("\n#{0} 인스타그램 사진 저장 실패!".format(choice_menu))
        error=1

    #해시태그 수집

    #첫번째 게시물 클릭
    def start(driver):
        driver.find_element_by_css_selector("div._9AhH0").click()
        time.sleep(3)

    start(driver)

    #게시물의 내용중에 해시태그만 수집
    def get_tag(driver):
        html = driver.page_source
        soup = BeautifulSoup(html, 'lxml')
        try:
            content = soup.select('div.C4VMK > span')[0].text
        except:
            content = ' '
        tags = re.findall('#[A-Za-z0-9가-힣]+', content) 
        tag = ''.join(tags).replace("#"," ") # "#" 제거 
        tag_data = tag.split() 
        return tag_data

    #화살표 모양눌러서 다음 게시물로 이동
    def move_next(driver):
        driver.find_element_by_css_selector ('a.coreSpriteRightPaginationArrow').click()
        time.sleep(3)

    #게시물이 n개 이상이면, n개 게시물의 해시태그 크롤링.
    #게시물이 n개 미만이면, 게시물 개수만큼 해시태그 크롤링
    insta_tag = []
    if data_num > 30:
        count = 30
    else:
        count = data_num

    for i in range(count-1): 
        try:
            tags = get_tag(driver)    
            for tag_one in tags: 
                insta_tag.append(tag_one)
            move_next(driver)
        except:
            time.sleep(2)
            move_next(driver)
    print()  
    print("< #{}관련 게시물 해시태그 >".format(choice_menu))
    print(insta_tag[:])
    
    insta_tag=(" ").join(insta_tag) #해시태그 요소들을 하나의 문자열로 만듬 
    wordcloud = WordCloud(background_color='white').generate(insta_tag)

    wordcloud = WordCloud(
        font_path = "C:\\Users\\KSY\\Desktop\\웹파\\프로젝트\\My_TermProject\\NanumFontSetup_TTF_BARUNPEN\\NanumBarunpenB.ttf", #폰트경로
        width = 800,
        height = 800
    )

    print("\n< #{0} 워드클라우드 >".format(choice_menu))
    wordcloud = wordcloud.generate(insta_tag)
    fig = plt.figure(figsize=(30,12))
    plt.imshow(wordcloud)
    plt.axis("off") #x,y축 숫자제거
    plt.savefig('./hashtag_wordcloud/'+ choice_menu) #hashtag_wordcloud폴더에 저장
    plt.show()
    print()
    print("\n#{0} 워드클라우드 사진 저장 완료!".format(choice_menu))
except:
    if error==0: #게시물 페이지가 존재하지 않거나 검색량이 아예 뜨지않을때
        print("죄송합니다. 검색하신 해시태그 페이지를 찾을수 없습니다.")
    elif error==1:  #게시물 검색량이 0일때 
        print("죄송합니다. 검색하신 해시태그 게시물이 없습니다.")
finally:
    driver.close()




1. 스타벅스 음료 
2. 스타벅스 케이크
찾고싶은 정보 번호 입력 >1

*** 스타벅스 메뉴목록 ***

1.['나이트로바닐라크림']
2.['나이트로쇼콜라클라우드']
3.['나이트로콜드브루']
4.['돌체콜드브루']
5.['바닐라크림콜드브루']
6.['제주비자림콜드브루']
7.['콜드브루']
8.['콜드브루몰트']
9.['콜드브루플로트']
10.['크리스마스콜드브루콘파나']
11.['아이스커피']
12.['오늘의커피']
13.['블론드토피넛라떼']
14.['아이스블론드토피넛라떼']
15.['아이스토피넛라떼']
16.['토피넛라떼']
17.['에스프레소콘파나']
18.['에스프레소마키아또']
19.['아이스카페아메리카노']
20.['카페아메리카노']
21.['아이스카라멜마키아또']
22.['카라멜마키아또']
23.['아이스카푸치노']
24.['카푸치노']
25.['라벤더카페브레베']
26.['럼샷코르타도']
27.['리저브바닐라빈라떼']
28.['리저브아이스바닐라빈라떼']
29.['스타벅스돌체라떼']
30.['시나몬롤돌체라떼']
31.['아이스라벤더카페브레베']
32.['아이스스타벅스돌체라떼']
33.['아이스시나몬롤돌체라떼']
34.['아이스제주샤이닝바나나라떼']
35.['아이스카페라떼']
36.['제주샤이닝바나나라떼']
37.['카페라떼']
38.['아이스카페모카']
39.['아이스화이트초콜릿모카']
40.['카페모카']
41.['화이트초콜릿모카']
42.['바닐라플랫화이트']
43.['바닐라스타벅스더블샷']
44.['블론드바닐라더블샷마키아또']
45.['아이스블론드바닐라더블샷마키아또']
46.['에스프레소']
47.['커피스타벅스더블샷']
48.['클래식아포가토']
49.['헤이즐넛스타벅스더블샷']
50.['토피넛팝콘트리프라푸치노']
51.['다크초콜릿블렌디드']
52.['더블에스프레소칩프라푸치노']
53.['모카프라푸치노']
54.['에스프레소프라푸치노']
55.['자바칩프라푸치노']
56.['카라멜프라푸치노']
57.['화이트초콜릿모카프라푸치노']
